In [1]:
import cvxpy as cp
import numpy as np
from Enhrenfest_methods import *
np.random.seed(22)

In [2]:
''' Now I want to create the following parameter for each type of agents

    - potential_reward_lst: reward rate
    - mu_lst: intensity under the active phase
    - lambda_lst: intensity under the passive phase

    - num_types: The number of types
    - scale: Adjust the numerical scale of intensity
'''

def sample_mu_values(num_types, scale = 10):
    # Sample mus
    mus = np.random.uniform(0, 1, num_types) * scale
    return list(mus)

def sample_lambda_values(num_types, scale = 10):
    # Sample mus
    lambdas = np.random.uniform(0, 1, num_types) * scale
    return list(lambdas)

In [3]:
## This generates the normal MDP process
def generate_transition_matrix_with_dummy_states(num_arms, num_states, num_dummy_states, mu_lst, lambda_lst, time_step):
    # Initialize transition matrices
    P = np.zeros((num_arms, num_states + num_dummy_states, 2, num_states + num_dummy_states))
    K = num_states - 1

    for n in range(num_arms):
        for s in range(num_states):
            if s == 0:
                # Boundary condition for state 0 (active)
                P[n, s, 1, s + num_states] = 1

                P[n, s, 0, s + 1] = lambda_lst[n] * (K - s) * time_step
                P[n, s, 0, s] = 1 - (lambda_lst[n] * (K - s) * time_step)
            if s == num_states - 1:
                # Boundary condition for state 9 (passive)
                P[n, s, 0, s] = 1

                P[n, s, 1, s - 1 + num_states] = mu_lst[n] * s * time_step
                P[n, s, 1, s + num_states] = 1 - mu_lst[n] * s * time_step

            else:
                P[n, s, 0, s + 1] = lambda_lst[n] * (K - s) * time_step
                P[n, s, 0, s] = 1 - lambda_lst[n] * (K - s) * time_step
                P[n, s, 1, s - 1 + num_states] = mu_lst[n] * s * time_step
                P[n, s, 1, s + num_states] = 1 - mu_lst[n] * s * time_step
        # Transitions within dummy states (s in S_a)
        for s_a in range(num_states, num_states + num_dummy_states):
            normal_state_index = s_a - num_states
            # Copy transitions from the corresponding normal state for action 0 within dummy states
            P[n, s_a, 0, num_states:num_states + num_dummy_states] = P[n, normal_state_index, 0, 0:num_states]
            P[n, s_a, 1, num_states:num_states + num_dummy_states] = P[n, normal_state_index, 0, 0:num_states]

    return P

In [4]:
S = 10
N = 10
group_member_num = 3
K = 3
S_prime = S * 2
A = 2
T = 10
num_simulations = 100
time_step = 0.01
mu_lst = sample_mu_values(N)
lambda_lst = sample_lambda_values(N)
potential_reward_lst = np.random.uniform(1, 10, N)

In [5]:
# change here to generate different types of transition matrices
P = generate_transition_matrix_with_dummy_states(N, S, S, mu_lst, lambda_lst, time_step)

In [6]:
optimal_reward, fluid_reward, fluid_std = fluid_policy(N, T, S, S_prime, A, K, group_member_num, P, \
                 num_simulations, potential_reward_lst, time_step)

/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


Solver status: optimal
Optimal value: 7.217832258648182
Average total reward over 100 simulations: 20.533731565208, stdev is 2.282166717147908
Number of pulls in dummy states (LP-based policy): 0


In [6]:
random_reward, random_std = random_policy(N, T, S, S_prime, A, K, group_member_num, P, \
                 num_simulations, potential_reward_lst, time_step)

Average total achieved value over 100 simulations (Random policy): 104.34206378140925, stdev is 8.06050257135824
